# Microsoft Malware Prediction Competition
# How To Score Over 0.702 LB with LGBM!
During the Microsoft Malware Prediction competition, everyone wanted to know how to score over 0.700 LB with a single LGBM. Since the competition is over, we can now discuss the secret 12 engineered features! This model scores 0.702 public LB and 0.775 private LB with a single LGBM. (As part of an ensemble, it scores 0.707 LB [here][2].)

# Why did this model come in 1435th place?
The private dataset contains 33% outliers. Those outliers ruin every model's score. This is explained [here][1]. When those outliers are removed, this model scores an amazing 0.775 AUC !!  
  
Even if you fit the clean data (sine wave pattern below) perfectly, the presence of outliers adds so much error that your model's AUC is terrible. See model one below. The presence of outliers hurt the most accurate solutions the most. If your solution only fits the data (sine wave pattern) loosely, then you have a chance of being near the outliers and may receive a better private LB score as seen in model two below.
  
  
![image](http://playagricola.com/Kaggle/errorone31919.png)  
  
![image](http://playagricola.com/Kaggle/errortwo31919.png)

# Load Data and Fix Pandas Memory Leak  
  
[1]: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/84745
[2]: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/84135

In [1]:
# monkeypatches.py

# Solving memory leak problem in pandas
# https://github.com/pandas-dev/pandas/issues/2659#issuecomment-12021083
import pandas as pd, sys
from ctypes import cdll, CDLL
try:
    cdll.LoadLibrary("libc.so.6")
    libc = CDLL("libc.so.6")
    libc.malloc_trim(0)
except (OSError, AttributeError):
    libc = None

__old_del = getattr(pd.DataFrame, '__del__', None)

def __new_del(self):
    if __old_del:
        __old_del(self)
    libc.malloc_trim(0)

if libc:
    print('Applying monkeypatch for pd.DataFrame.__del__', file=sys.stderr)
    pd.DataFrame.__del__ = __new_del
else:
    print('Skipping monkeypatch for pd.DataFrame.__del__: libc or malloc_trim() not found', file=sys.stderr)

Applying monkeypatch for pd.DataFrame.__del__


In [2]:
# SET THIS VARIABLE TO TRUE TO RUN KERNEL QUICKLY AND FIND BUGS
# ONLY 10000 ROWS OF DATA IS LOADED
Debug = False

import numpy as np, pandas as pd, gc, random
import matplotlib.pyplot as plt

def load(x):
    ignore = ['MachineIdentifier']
    if x in ignore: return False
    else: return True

# LOAD TRAIN AND TEST
if Debug:
    df_train = pd.read_csv('../input/microsoft-malware-prediction/train.csv',dtype='category',usecols=load,nrows=10000)
else:
    df_train = pd.read_csv('../input/microsoft-malware-prediction/train.csv',dtype='category',usecols=load, nrows=1000000)
df_train['HasDetections'] = df_train['HasDetections'].astype('int8')
if 5244810 in df_train.index:
    df_train.loc[5244810,'AvSigVersion'] = '1.273.1144.0'
    df_train['AvSigVersion'].cat.remove_categories('1.2&#x17;3.1144.0',inplace=True)

if Debug:
    df_test = pd.read_csv('../input/microsoft-malware-prediction/test.csv',dtype='category',usecols=load,nrows=10000)
else:
    df_test = pd.read_csv('../input/microsoft-malware-prediction/test.csv',dtype='category',usecols=load, nrows=1000000)
    
print('Loaded',len(df_train),'rows of TRAIN and',len(df_test),'rows of TEST')

Loaded 1000000 rows of TRAIN and 1000000 rows of TEST


# Define Encoding Functions

In [3]:
# FREQUENCY ENCODE SEPARATELY
def encode_FE(df,col):
    vc = df[col].value_counts(dropna=False, normalize=True).to_dict()
    nm = col+'_FE'
    df[nm] = df[col].map(vc)
    df[nm] = df[nm].astype('float32')
    return [nm]

# FREQUENCY ENCODE TOGETHER
def encode_FE2(df1, df2, col):
    df = pd.concat([df1[col],df2[col]])
    vc = df.value_counts(dropna=False, normalize=True).to_dict()
    nm = col+'_FE2'
    df1[nm] = df1[col].map(vc)
    df1[nm] = df1[nm].astype('float32')
    df2[nm] = df2[col].map(vc)
    df2[nm] = df2[nm].astype('float32')
    return [nm]

# FACTORIZE
def factor_data(df_train, df_test, col):
    df_comb = pd.concat([df_train[col],df_test[col]],axis=0)
    df_comb,_ = df_comb.factorize(sort=True)
    # MAKE SMALLEST LABEL 1, RESERVE 0
    df_comb += 1
    # MAKE NAN LARGEST LABEL (need to remove attype('str') above)
    df_comb = np.where(df_comb==0, df_comb.max()+1, df_comb)
    df_train[col] = df_comb[:len(df_train)]
    df_test[col] = df_comb[len(df_train):]
    del df_comb
    
# OPTIMIZE MEMORY
def reduce_memory(df,col):
    mx = df[col].max()
    if mx<256:
            df[col] = df[col].astype('uint8')
    elif mx<65536:
        df[col] = df[col].astype('uint16')
    else:
        df[col] = df[col].astype('uint32')
        
# REDUCE CATEGORY CARDINALITY
def relax_data(df_train, df_test, col):
    cv1 = pd.DataFrame(df_train[col].value_counts().reset_index().rename({col:'train'},axis=1))
    cv2 = pd.DataFrame(df_test[col].value_counts().reset_index().rename({col:'test'},axis=1))
    cv3 = pd.merge(cv1,cv2,on='index',how='outer')
    factor = len(df_test)/len(df_train)
    cv3['train'].fillna(0,inplace=True)
    cv3['test'].fillna(0,inplace=True)
    cv3['remove'] = False
    cv3['remove'] = cv3['remove'] | (cv3['train'] < len(df_train)/10000)
    cv3['remove'] = cv3['remove'] | (factor*cv3['train'] < cv3['test']/3)
    cv3['remove'] = cv3['remove'] | (factor*cv3['train'] > 3*cv3['test'])
    cv3['new'] = cv3.apply(lambda x: x['index'] if x['remove']==False else 0,axis=1)
    cv3['new'],_ = cv3['new'].factorize(sort=True)
    cv3.set_index('index',inplace=True)
    cc = cv3['new'].to_dict()
    df_train[col] = df_train[col].map(cc)
    reduce_memory(df_train,col)
    df_test[col] = df_test[col].map(cc)
    reduce_memory(df_test,col)
    
# DISPLAY MEMORY STATISTICS
def display_memory(df_train, df_test):
    print(len(df_train),'rows of training data use',df_train.memory_usage(deep=True).sum()//1e6,'Mb memory!')
    print(len(df_test),'rows of test data use',df_test.memory_usage(deep=True).sum()//1e6,'Mb memory!')

# CONVERT TO CATEGORIES
def categorize(df_train, df_test, cols):
    for col in cols:
        df_train[col] = df_train[col].astype('category')
        df_test[col] = df_test[col].astype('category')

# Feature Engineering
Here is half of the secret sauce. You must engineer the following five variables. (The other half is frequency encoding 7 variables described below.). These variables were discovered by trying hundreds of engineered variables to see which ones increased Time Split Validation (TSV explained [here][1]). Each variable was added to the model one at a time and validation score was recorded. Only the follow 5 variables increased validation score.

* `AppVersion2` indicates whether your Windows Defender is up to date. This is the second number from AppVersion. Regardless of your operating system and version, you can always have AppVersion with second number equal 18. For example, you should have `4.18.1807.18075` instead of `4.12.xx.xx`.

* `Lag1` is the difference between AvSigVersion_Date and Census_OSVersion_Date. Since AvSigVersion is the virus definitions for Windows Defender, this variable indicates whether Windows Defender is out-of-date by comparing it's last install with the date of the operating system. Out-of-date antivirus indicates that a user either has better antivirus or they don't use their computer often. In either case, they have less HasDetections.

* `Lag5` is the difference between AvSigVersion_Date and July 26, 2018. The first observation in Microsoft's training data is July 26, 2018. Therefore if a computer has AvSigVersion_Date before this then their antivirus is out-of-date. (The first observation in the test data is September 27, so you use this difference when encoding the test data.)

* `driveA` is the ratio of harddrive partition used for the operating system with the total hard drive. Savy users install multiple operating systems and have a lower ratio. Savy users have reduced HasDetections.

* `driveB` is the difference between harddrive partition used for the operating system and total hard drive. Responsible users manager their hard drives well. Responsible users have reduced HasDetections.  
  
[1]: https://www.kaggle.com/cdeotte/time-split-validation-malware-0-68

In [4]:
from datetime import datetime, date, timedelta

# AS timestamp
datedictAS = np.load('../input/malware-timestamps/AvSigVersionTimestamps.npy')[()]
df_train['DateAS'] = df_train['AvSigVersion'].map(datedictAS)
df_test['DateAS'] = df_test['AvSigVersion'].map(datedictAS)

# OS timestamp
datedictOS = np.load('../input/malware-timestamps-2/OSVersionTimestamps.npy')[()]
df_train['DateOS'] = df_train['Census_OSVersion'].map(datedictOS)
df_test['DateOS'] = df_test['Census_OSVersion'].map(datedictOS)

# ENGINEERED FEATURE #1
df_train['AppVersion2'] = df_train['AppVersion'].map(lambda x: np.int(x.split('.')[1]))
df_test['AppVersion2'] = df_test['AppVersion'].map(lambda x: np.int(x.split('.')[1]))

# ENGINEERED FEATURE #2
df_train['Lag1'] = df_train['DateAS'] - df_train['DateOS']
df_train['Lag1'] = df_train['Lag1'].map(lambda x: x.days//7)
df_test['Lag1'] = df_test['DateAS'] - df_test['DateOS']
df_test['Lag1'] = df_test['Lag1'].map(lambda x: x.days//7)

# ENGINEERED FEATURE #3
df_train['Lag5'] = datetime(2018,7,26) - df_train['DateAS']
df_train['Lag5'] = df_train['Lag5'].map(lambda x: x.days//1)
df_train.loc[ df_train['Lag5']<0, 'Lag5' ] = 0
df_test['Lag5'] = datetime(2018,9,27) - df_test['DateAS'] #PUBLIC TEST
df_test['Lag5'] = df_test['Lag5'].map(lambda x: x.days//1)
df_test.loc[ df_test['Lag5']<0, 'Lag5' ] = 0
df_train['Lag5'] = df_train['Lag5'].astype('float32') # allow for NAN
df_test['Lag5'] = df_test['Lag5'].astype('float32') # allow for NAN

# ENGINEERED FEATURE #4
df_train['driveA'] = df_train['Census_SystemVolumeTotalCapacity'].astype('float')/df_train['Census_PrimaryDiskTotalCapacity'].astype('float')
df_test['driveA'] = df_test['Census_SystemVolumeTotalCapacity'].astype('float')/df_test['Census_PrimaryDiskTotalCapacity'].astype('float')
df_train['driveA'] = df_train['driveA'].astype('float32') 
df_test['driveA'] = df_test['driveA'].astype('float32') 

# ENGINNERED FEATURE #5
df_train['driveB'] = df_train['Census_PrimaryDiskTotalCapacity'].astype('float') - df_train['Census_SystemVolumeTotalCapacity'].astype('float')
df_test['driveB'] = df_test['Census_PrimaryDiskTotalCapacity'].astype('float') - df_test['Census_SystemVolumeTotalCapacity'].astype('float')
df_train['driveB'] = df_train['driveB'].astype('float32') 
df_test['driveB'] = df_test['driveB'].astype('float32') 

cols6=['Lag1']
cols8=['Lag5','driveB','driveA']

del df_train['DateAS'], df_train['DateOS'] #, df_train['DateBL']
del df_test['DateAS'], df_test['DateOS'] #, df_test['DateBL']
del datedictAS, datedictOS
x=gc.collect()

# Frequency Engineering
This is the second half of the secret sauce. There is a correlation between the frequency of time variables (`EngineVersion`, `AvSigVersion`, `AppVersion`, `Census_OSVersion`, `Census_OSBuildRevision`) and `HasDetections`. This occurs for two reasons explained [here][1]. One reason is a leak and will lead to overfitting train and public test because I believe that private test is a random sample and therefore does not benefit from the leak. The second reason is because out-of-date antivirus produces a front tail with both reduced frequency and reduced HasDetections (explained and verified [here][1]). To take advantage of these two correlations, you need to frequency encode the time variables **separately** for train and test. (Also since EngineVersion and AppVersion have low cardinality, I found that frequency encoded them doesn't increase CV or LB.)  

There is an actual correlation between frequency of `CountryIdentifier` and `HasDetections`. This is probably because popular countries are targeted. Also there is a true correlation between frequency of `Census_InternalBatteryNumberOfCharges` and `HasDetections`. I have no idea why this is the case. To take advantage of these correlations, frequency encode these variables in train and test **together**. 
  
These frequency engineered variables were discovered by trying every variable as FE encoded **separately** and FE encoded **together** to see which ones increased Time Split Validation (TSV explained [here][1]). Each variable was added to the model one at a time and validation score was recorded. Only the follow 7 FE variables increased validation score.
  
[1]: https://www.kaggle.com/c/microsoft-malware-prediction/discussion/84287

In [5]:
cols3 = []
# ENGINEERED FEATURES #6, #7, #8, #9, #10
FE = ['Census_OSVersion', 'Census_OSBuildRevision', 'Census_InternalBatteryNumberOfCharges', 'AvSigVersion', 'Lag1']
for col in FE:
    cols3 += encode_FE(df_train, col)
    encode_FE(df_test, col)
    
# ENGINEERED FEATURES #11, #12
FE2 = ['CountryIdentifier', 'Census_InternalBatteryNumberOfCharges']
for col in FE2:
    cols3 += encode_FE2(df_train, df_test, col)

# Categorical versus Numerical Encode
All 82 explanatory variables except `Census_PrimaryDiskTotalCapacity` and `Census_SystemVolumeTotalCapacity` are actually categorical and not numerical. However if you input them into LGBM as categorical, since their cardinality is so high, it leads to overfitting. Therefore validation shows that it is best to label encode variables (`pd.factorize(df[col],sort=True)`) into ordered integers and input them into LGBM as integers. Then set `num_leaves=2048` and LGBM has no trouble finding the categories anyway.  
  
After every variable was changed to dtype integer, each variable was tested one by one to see if making it categorical increases LGBM validation score. It was determined that the following 6 variables increased validation score. By making them categorical you prioritize them. So presumably these are important variables. Notice that they cover the variety of flavors of features. We have two geographical variables, one hardware variable, two software/virus variables, and one miscellenous. Flavors of variables are explained [here][1] called groupings.  
  
[1]: https://www.kaggle.com/cdeotte/embeddings-network-malware-0-697-0-773

In [6]:
CE = ['CountryIdentifier', 'SkuEdition', 'Firewall', 'Census_ProcessorCoreCount', 'Census_OSUILocaleIdentifier', 'Census_FlightRing']

# Remove Variables
Every variable was removed one at a time and validation score was recorded. Validation verified that we can remove the following 18 variables without decreasing model accuracy. `MachineIdentifier`, `ProductName`, `IsBeta`, `IsSxsPassiveMode`, `HasTpm`, `AutoSampleOptIn`, `PuaMode`, `UacLuaenable`, `Census_DeviceFamily`, `Census_ProcessorClass`, `Census_IsPortableOperatingSystem`, `Census_IsFlightsDisabled`, `Census_IsVirtualDevice`, `Census_OSSkuName`, `OsVer`, `Census_OSArchitecture`, `Census_OSInstallLanguageIdentifier`, `SMode`. The first one is obvious. The next 12 have more than 98% of their data in one category value. The next 4 correlate with another variable greater than `r=0.99`. And the last variable `SMode` changes is distribution radically over time and leads to decreased CV and LB. The other variables don't decrease validation per say but they don't increase it either, so we remove them.

In [7]:
cols = [x for x in df_train.columns if x not in ['HasDetections']+CE+cols3+cols6+cols8]
cols2 = CE; ct = 1
    
for col in cols.copy():
    rate = df_train[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.98:
        del df_train[col]
        del df_test[col]
        cols.remove(col)
        ct += 1

rmv3=['Census_OSSkuName', 'OsVer', 'Census_OSArchitecture', 'Census_OSInstallLanguageIdentifier']
rmv4=['SMode']
for col in rmv3+rmv4:
    del df_train[col]
    del df_test[col]
    cols.remove(col)
    ct +=1
    
print('Removed',ct,'variables')
x=gc.collect()

Removed 18 variables


# Reduce Feature Cardinality
Many variables have over 1000 category values!! We must reduce cardinality. We will remove any category value that contains 0.01% or less of total data. Additionally to prevent overfitting the train data, we will remove any category value that appears 4x more frequenly in train than test. And we will remove any category value that appears 4x less frequency in train than test. Explained [here][1].
  
For example, `EngineVersion` has many category values like `1.1.15100.1`, `1.1.15200.1`, `1.1.15300.6`, etc. The category value `1.1.15300.6` appears 4x more frequently in test than train, therefore we will remove it. We "remove it" by changing that computer's `EngineVersion` to a new category value of `TrashCan`.
  
[1]: https://www.kaggle.com/cdeotte/time-split-validation-malware-0-68

In [8]:
print('Factorizing...')
for col in cols+cols2+cols6:
    factor_data(df_train, df_test, col)
print('Relaxing data...')
for col in cols+cols2: relax_data(df_train, df_test, col)
print('Optimizing memory...')
for col in cols+cols2+cols6:
    reduce_memory(df_train, col)
    reduce_memory(df_test, col)
# Converting 6 variables to categorical
categorize(df_train, df_test, cols2)
    
print('Number of variables is',len(cols+cols2+cols3+cols6+cols8))
display_memory(df_train, df_test)

Factorizing...
Relaxing data...
Optimizing memory...
Number of variables is 76
1000000 rows of training data use 115.0 Mb memory!
1000000 rows of test data use 114.0 Mb memory!


In [9]:
microsoft_100w_train_data = df_train[cols+cols2+cols3+cols6+cols8]
microsoft_100w_train_label = df_train['HasDetections']

In [10]:
microsoft_100w_train_data.to_csv("/kaggle/working/microsoft_100w_train_data.csv", index=False)
microsoft_100w_train_label.to_csv("/kaggle/working/microsoft_100w_train_label.csv", index=False)